In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision.transforms import v2
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

batch_size_ = 256
num_epochs_ = 50
learning_rate_ = 0.005
transforms_ = v2.Compose(
    [
        v2.ToTensor(),
        v2.Normalize(
            mean=[
                0.5,
            ],
            std=[
                0.5,
            ],
        ),
    ]
)
"""
FashionMNIST数据集,包含10个类别的图像,每个类别有6000张训练图像和1000张测试图像
    训练集:60,000张图像
    测试集:10,000张图像
    训练集和测试集的图像大小为28x28像素,每个像素点是一个灰度值,范围在0到1之间
    训练集和测试集的图像标签为数字,范围在0到9之间
"""
train_dataset_ = datasets.FashionMNIST(
    root="../data", train=True, download=True, transform=transforms_
)

test_dataset_ = datasets.FashionMNIST(
    root="../data", train=False, download=True, transform=transforms_
)


train_loader_ = DataLoader(
    dataset=train_dataset_, shuffle=True, num_workers=4, batch_size=batch_size_
)
test_loader_ = DataLoader(
    dataset=test_dataset_, shuffle=True, num_workers=4, batch_size=batch_size_
)


class SoftmaxRegresion(nn.Module):
    def __init__(self, input_dim=28 * 28, output_dim=10):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        logits = self.linear(x)
        return F.softmax(logits, dim=1)


model_ = SoftmaxRegresion()
criterion_ = nn.CrossEntropyLoss()
optimizer_ = optim.SGD(model_.parameters(), lr=learning_rate_)

writer = SummaryWriter("../logdir/softmax_regression")
dummy_input_ = torch.randn(1, 1, 28, 28)
writer.add_graph(model_, dummy_input_)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_.to(device)
tran_losses = []
test_accuracies = []

for epoch in range(num_epochs_):
    model_.train()
    total_loss = 0.0
    correct_ = 0
    total_ = 0
    for i, (images, labels) in enumerate(train_loader_):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_(images)
        loss = criterion_(outputs, labels)
        optimizer_.zero_grad()
        loss.backward()
        optimizer_.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_ += labels.size(0)
        correct_ += (predicted == labels).sum().item()
        if i % 100 == 0:
            img_grid = torchvision.utils.make_grid(images)
            writer.add_image(
                "trainning_images", img_grid, global_step=epoch * len(train_loader_) + i
            )
    avg_loss = total_loss / len(train_loader_)
    train_acc_ = 100 * correct_ / total_
    writer.add_scalar("Loss/train", avg_loss, epoch)
    writer.add_scalar("Accuracy/train", train_acc_, epoch)

    model_.eval()
    test_correct_ = 0
    test_total_ = 0
    with torch.no_grad():
        for images, labels in test_loader_:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model_(images)
            _, predicted = torch.max(outputs.data, 1)
            test_total_ += labels.size(0)
            test_correct_ += (predicted == labels).sum().item()
    test_acc_ = 100 * test_correct_ / test_total_
    writer.add_scalar("Accuracy/test", test_acc_, epoch)
    print(
        f"Epoch [{epoch+1}/{num_epochs_}], Loss: {avg_loss:.4f}, Train Acc: {train_acc_:.2f}%, Test Acc: {test_acc_:.2f}%"
    )
    # 添加分类标签名称
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
writer.add_text('Class Labels', str(class_names))
# 添加权重分布直方图
for name, param in model_.named_parameters():
    writer.add_histogram(name, param, epoch)

writer.close()

Epoch [1/50], Loss: 2.2144, Train Acc: 32.06%, Test Acc: 40.95%
Epoch [2/50], Loss: 2.0549, Train Acc: 48.35%, Test Acc: 56.03%
Epoch [3/50], Loss: 1.9358, Train Acc: 63.09%, Test Acc: 65.93%
Epoch [4/50], Loss: 1.8773, Train Acc: 66.65%, Test Acc: 67.08%
Epoch [5/50], Loss: 1.8432, Train Acc: 68.72%, Test Acc: 70.18%
Epoch [6/50], Loss: 1.8159, Train Acc: 71.21%, Test Acc: 71.44%
Epoch [7/50], Loss: 1.7992, Train Acc: 72.22%, Test Acc: 72.32%
Epoch [8/50], Loss: 1.7869, Train Acc: 72.92%, Test Acc: 72.88%
Epoch [9/50], Loss: 1.7771, Train Acc: 73.43%, Test Acc: 73.56%
Epoch [10/50], Loss: 1.7693, Train Acc: 73.83%, Test Acc: 73.88%
Epoch [11/50], Loss: 1.7631, Train Acc: 74.21%, Test Acc: 74.20%
Epoch [12/50], Loss: 1.7576, Train Acc: 74.55%, Test Acc: 74.46%
Epoch [13/50], Loss: 1.7529, Train Acc: 74.82%, Test Acc: 74.75%
Epoch [14/50], Loss: 1.7488, Train Acc: 75.08%, Test Acc: 74.91%
Epoch [15/50], Loss: 1.7450, Train Acc: 75.31%, Test Acc: 75.10%
Epoch [16/50], Loss: 1.7414, Train